In [1]:
##tools
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [ ]:
##Creating Arxiv tool
api_arxiv_wrapper = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_arxiv_wrapper, description="Query Arxiv Papers")
print(arxiv.name)

arxiv


In [3]:
##Testing Arxiv to fetch research papers
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [ ]:
##Creating Wikipedia tool
api_wiki_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wiki_wrapper, description="Fetcing from Wiki")
wiki.name

'wikipedia'

In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [7]:
##creatin Tavily tool for searching from web
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults()

C:\Users\Jamadagni\AppData\Local\Temp\ipykernel_10936\324114086.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [8]:
##calling web 
tavily.invoke("Recent news on AI")

[{'title': 'The latest AI news we announced in October - The Keyword',
  'url': 'https://blog.google/technology/ai/google-ai-updates-october-2025/',
  'content': '"The Latest AI News We Announced in October" highlights Google\'s recent AI advancements.\n Google\'s AI breakthroughs include a quantum algorithm faster than supercomputers.\n An AI model called Cell2Sentence-Scale may offer a new cancer therapy pathway.\n Gemini Enterprise is now the "front door" for Google AI in the workplace.\n New AI security features protect against scams, plus Google Home gets AI upgrades.\n\n## Shakespeare-ish [...] # The latest AI news we announced in October\n\nNov 04, 2025\n\nHere’s a recap of some of our biggest AI updates from October, including Gemini Enterprise, an AI model to accelerate cancer research and a big quantum computing breakthrough.\n\nSuperG\n\n## General summary [...] In October Google made advancements in AI to improve daily life. Google introduced Gemini for Home and vibe coding

In [12]:
##Combine all these tools in the list
tools = [arxiv, wiki, tavily]

In [10]:
##Initialize the LLM model
from langchain_groq import ChatGroq
llm =ChatGroq(model="qwen/qwen3-32b")

In [11]:
llm.invoke("who is pm of india")

AIMessage(content="<think>\nOkay, so I need to find out who the current Prime Minister of India is. Let me start by recalling what I know. India is a country in South Asia, and its government is a federal parliamentary democratic republic. The Prime Minister is the head of government and the head of the Council of Ministers. \n\nI remember that the Prime Minister is usually the leader of the majority party in the Lok Sabha, which is the lower house of India's Parliament. The last time I checked, Narendra Modi was the Prime Minister. He was the leader of the Bharatiya Janata Party (BJP), which is a right-wing party. His party has been in power since 2014, so it's possible he's still in office.\n\nBut wait, when was the last election? The general elections in India are held every five years unless the government is dissolved earlier. The last election was in 2019, so the next one should be in 2024. That means Modi would be in office until then. Unless there was a by-election or a party c

In [13]:
## to bind the list of tools to our current LLm
llm_with_tools = llm.bind_tools(tools=tools)

In [14]:
## Execute the call or query using LLM which will use the available tools based on the query
llm_with_tools.invoke("what is the latest newson AI")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the latest news on AI. Let me see which tool to use here. They mentioned "latest news," so I need a source that provides current information. The available tools are arxiv, wikipedia, and tavily_search_results_json.\n\nArxiv is for academic papers, which might not be the best for news. Wikipedia is good for general information but might not have the most up-to-date news. The tavily_search_results_json is described as a search engine optimized for current events, so that seems like the right choice. I should use that to get the latest news on AI. The query parameter should be "latest news on AI" to fetch relevant results.\n', 'tool_calls': [{'id': 'j775058wn', 'function': {'arguments': '{"query":"latest news on AI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 322, 'total_tokens': 493, 'completion_tim

In [15]:
llm_with_tools.invoke("what is the latest research on Quantum computing")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking about the latest research on quantum computing. I need to figure out which tool to use here. The available tools are arxiv, wikipedia, and tavily_search.\n\nArxiv is great for academic papers, so that\'s probably the best place to look for the latest research. Wikipedia might have general information but not the most recent studies. Tavily search is for current events, but quantum computing research is more likely to be found in academic databases first.\n\nSo I should use the arxiv function. The parameter needed is a query. The user\'s query is "latest research on Quantum computing". I\'ll need to format that into the arxiv function call. Let me make sure the query is specific enough. Maybe "quantum computing" as the search term. Yep, that should work. Let me structure the tool call accordingly.\n', 'tool_calls': [{'id': 't6564ewma', 'function': {'arguments': '{"query":"quantum computing"}', 'name'

Workflow
